In [92]:
from collections import Counter
import nltk
import requests

Buscar um conjunto de discursos de exemplo:

In [93]:
json = requests.get('http://babel.labhackercd.leg.br/api/v1/manifestations?manifestation_type__id=2').json()
speeches = [data['content'] for data in json['results']]
speeches

['Sr. Presidente, caros Deputados, população brasileira que nos acompanha pela   TV Câmara   , hoje é um dia muito importante para o nosso País, pois é um dia histórico.    Isso a que nós estamos assistindo é estarrecedor! Este já é o mais longo processo de cassação vivido nesta Casa. Foram muitas as manobras e muitas as tentativas de evitar a cassação do mandato do Deputado Eduardo Cunha.    Nós estamos aqui hoje para defender a cassação do mandato do Deputado Eduardo Cunha, porque ele fez muito mal ao Brasil. Além de todos os seus atos de corrupção e de tudo aquilo que foi denunciado, em nenhum momento o Deputado Eduardo Cunha apresentou algo que pudesse provar a sua inocência. Em vez disso, no Conselho de Ética e na CCJ o que o Deputado Eduardo Cunha fez foram ameaças a seus colegas, dizendo:   "Hoje sou eu. Amanhã serão vocês!"      De fato, Deputado Luiz Couto, o povo brasileiro e nós queremos saber quem são os outros Deputados que o Deputado Eduardo Cunha disse que também sofrerã

## Tokenize
Transforma o texto em uma lista de tokens.

In [94]:
def tokenize(text):
    return nltk.tokenize.word_tokenize(text, language='portuguese')

## Stemming
Processo de reduzir as palavras ao seu radical. Pode ser passado um dicionário de `Counters`, para poder manter a referência da palavra original.

In [95]:
def stemmize(token, stem_reference=None):
    token = token.casefold()
    stemmer = nltk.stem.RSLPStemmer()
    stemmed = stemmer.stem(token)

    if stem_reference is not None:
        reference = stem_reference.get(stemmed, Counter())
        reference.update([token])
        stem_reference[stemmed] = reference

    return stemmed

In [96]:
def stemmize_stopwords(stopwords=None):
    default_stopwords = nltk.corpus.stopwords.words('portuguese')
    if stopwords is not None:
        default_stopwords += stopwords

    return list(set([
        stemmize(stopword)
        for stopword in default_stopwords
    ]))


## Bag of Words (BOW)
Retorna um `Counter` com os `stems` de todos os termos que não estão listados na lista de stopwords. Também pode ser passado um parâmetro `method` para definir a forma que a frequência será computada (somente o método `frequency` foi implementado por enquanto). Além disso, também retorna um dicionário de `Counter`, como referencia dos termos não "stemmizados".

In [97]:
EXTRA_STOPWORDS = [',', '.', 'srs', 'sr.', 'sra.', 'deputado', 'presidente', 'é', ':', "''", '`', '!', '``', '?', 'nº', 's.a.']

def bow(text, method='frequency'):
    tokens = tokenize(text)
    stopwords = stemmize_stopwords(
        stopwords=EXTRA_STOPWORDS
    )
    stem_reference = {}

    text_bow = Counter([
        stemmize(token) for token in tokens
        if stemmize(token, stem_reference=stem_reference) not in stopwords
    ])
    return text_bow, stem_reference

## Most Common Words
Retorna uma lista de tuplas com os termos mais frequentes e suas frequencias. Pode receber como parâmtro a quantidade de termos.

In [98]:
def most_common_words(text_bow, reference, n=None):
    most_common = []

    for token in text_bow.most_common(n):
        stem, frequency = token

        # reference[stem] is a Counter and most_comon(1) return a list
        # of tuples: ('word', occurrences)
        word = reference[stem].most_common(1)[0][0]
        most_common.append((word, frequency))
    return most_common

In [ ]:
text_bow, reference = bow(' '.join(speeches))
common = most_common_words(text_bow, reference)
pprint(common[:50])

In [100]:
# Normalize common words
total = sum(text_bow.values(), 0.0)
for token in text_bow:
    text_bow[token] /= total
text_bow.most_common(20)

[('brasil', 0.014462809917355372),
 ('cunh', 0.008677685950413223),
 ('hoj', 0.007851239669421488),
 ('eduard', 0.007851239669421488),
 ('tod', 0.0070247933884297524),
 ('vot', 0.0070247933884297524),
 ('cas', 0.006818181818181818),
 ('govern', 0.006611570247933884),
 ('quer', 0.006198347107438017),
 ('direit', 0.005785123966942148),
 ('golp', 0.005371900826446281),
 ('cass', 0.005165289256198347),
 ('grand', 0.005165289256198347),
 ('trabalh', 0.004752066115702479),
 ('país', 0.004545454545454545),
 ('dia', 0.0043388429752066115),
 ('aqu', 0.0043388429752066115),
 ('faz', 0.0043388429752066115),
 ('outr', 0.004132231404958678),
 ('públic', 0.004132231404958678)]

In [101]:
from pprint import pprint

common = most_common_words(text_bow, reference)
pprint(common[:20])

[('brasil', 0.014462809917355372),
 ('cunha', 0.008677685950413223),
 ('hoje', 0.007851239669421488),
 ('eduardo', 0.007851239669421488),
 ('todos', 0.0070247933884297524),
 ('votar', 0.0070247933884297524),
 ('casa', 0.006818181818181818),
 ('governo', 0.006611570247933884),
 ('quero', 0.006198347107438017),
 ('direitos', 0.005785123966942148),
 ('golpe', 0.005371900826446281),
 ('cassação', 0.005165289256198347),
 ('grande', 0.005165289256198347),
 ('trabalho', 0.004752066115702479),
 ('país', 0.004545454545454545),
 ('dia', 0.0043388429752066115),
 ('aqui', 0.0043388429752066115),
 ('fazer', 0.0043388429752066115),
 ('outros', 0.004132231404958678),
 ('pública', 0.004132231404958678)]
